In [ ]:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold # import KFold
from sklearn import metrics

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/CA683')

Mounted at /content/drive/


In [ ]:
import matplotlib.pyplot as plt

data =pd.read_csv("bitcoin_and_sentiments_data.csv")
data.columns
data.fillna(inplace=True, value=0)

data.head(2)

#ax = plt.gca()
  
#use plot() method on the dataframe
#data.plot( x = 'time' , y = 'vader_sent_mean_by_hour', ax = ax )
#data.plot( x = 'time' , y = 'blob_sent_mean_by_hour' , ax = ax )


,Unnamed: 0,time,closed_price_by_hour,price_diff,movement,Daily_Weight_mean_by_hour,Daily_Weight_count_by_hour,blob_sent_mean_by_hour,subjectivity_mean_by_hour,vader_sent_mean_by_hour,tweet_diff,blob_sent_mean_by_hour_diff,vader_sent_mean_by_hour_diff,tweet_movement,blob_sent_movement,vader_sent_movement
0,0,2016-02-19 08:00:00,418.409535,1.745839,1,0.0,1,0.000000,0.15,-0.4019,0.0,-3.931830,-0.442529,1,0,0
1,1,2016-05-15 21:00:00,456.989825,-0.042136,0,1.0,1,0.136364,0.50,0.0000,0.0,0.136364,0.401900,1,1,1


In [ ]:
# vader = data["vader_sent_hourly mean"].values
vader = data["vader_sent_mean_by_hour"].values
# blob = data["blob_sent_hourly mean"].values
blob = data["blob_sent_mean_by_hour"].values
# tweetdir = data["tweet dir"].values
tweetdir = data["tweet_diff"].values
# tweet_vol = data["Daily Weight count"].values
tweet_vol = data["Daily_Weight_count_by_hour"].values
# v_move = data["vsent dir"].values
v_move = data["vader_sent_mean_by_hour_diff"].values #vader_sent_mean_by_hour diff
# b_move = data["bsent dir"].values
b_move = data["blob_sent_mean_by_hour_diff"].values #blob_sent_mean_by_hour diff
# diff = data['price mov']
diff = data['movement'].values # 5hr MA diff movement
# multifeat = data.filter(["vader_sent_hourly mean","blob_sent_hourly mean","tweet dir","Daily Weight count"])
multifeat = data.filter(["vader_sent_mean_by_hour","blob_sent_mean_by_hour","tweet dir","Daily_Weight_count_by_hour"])
# close = data['Close'].values
close = data['closed_price_by_hour'].values

In [ ]:
features = [multifeat,vader,blob,tweetdir,tweet_vol,v_move,b_move]

feature = ['multifeat','vader','blob','tweetdir','tweet_vol','v_move','b_move']

In [ ]:
def f1(cm):
    aNo = cm[0][0]
    ayes = cm[1][0]
    pno = cm[0][1]
    pyes = cm[1][1]
    

In [ ]:
#Kfold Cross Validation with Logistic Regression

for i in range(len(features)):
    if i >0:
        xbar = features[i].reshape(-1,1)
    else:
        xbar = features[i]
    kf = KFold(n_splits=10) # Define the split - into 2 folds 
    
    kf.get_n_splits(xbar) # returns the number of splitting iterations in the cross-validator
    #KFold(n_splits=10, random_state=None, shuffle=False)
    scores = []
    f1=[]
    mat = np.zeros([2,2])
    for train_index, test_index in kf.split(xbar):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = xbar[train_index[0]:train_index[1189]], xbar[test_index[0]:test_index[132]]
        y_train, y_test = diff[train_index[0]:train_index[1189]], diff[test_index[0]:test_index[132]]
        logisticRegr = LogisticRegression()
        logisticRegr.fit(X_train, y_train)

        predictions = logisticRegr.predict(X_test)
        score = logisticRegr.score(X_test, y_test)
        scores.append(score)
        #print("{} Score : {}".format(feature[i],score))
        cm = metrics.confusion_matrix(y_test, predictions)
        f1_score = metrics.f1_score(y_test,predictions)
        f1.append(f1_score)
        mat += cm
        
        #print(cm)
    idx = np.mean(scores)
    print("{} Score : {}".format(feature[i],round(idx, 3)))
    print(round(np.mean(f1), 3))
    print(mat)



multifeat Score : 0.538
0.698
[[  3. 604.]
 [  6. 707.]]
vader Score : 0.54
0.7
[[  0. 607.]
 [  0. 713.]]
blob Score : 0.537
0.697
[[  2. 605.]
 [  6. 707.]]
tweetdir Score : 0.539
0.698
[[  8. 599.]
 [  9. 704.]]
tweet_vol Score : 0.54
0.7
[[  0. 607.]
 [  0. 713.]]
v_move Score : 0.542
0.701
[[  2. 605.]
 [  0. 713.]]
b_move Score : 0.539
0.699
[[  3. 604.]
 [  4. 709.]]


In [ ]:
print(diff)

[1 0 1 ... 0 0 1]


In [ ]:
#multi feature logistic regression

features = [multifeat, vader,blob,tweetdir,tweet_vol]

feature = ['multifeat', 'vader','blob','tweet_dir','tweet_volume']

scores = []
f1=[]
mat = np.zeros([2,2])
for i in range(len(features)):
  df_label = pd.DataFrame(diff)
  if i == 0:
    X_train, X_test, y_train, y_test = train_test_split(multifeat, df_label, test_size=0.20, random_state=42)
  else:
    df_feature = pd.DataFrame(features[i])

    X_train, X_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=0.20, random_state=42)
    #print(y_train)
  
  logisticRegr = LogisticRegression()
  logisticRegr.fit(X_train, y_train)
  predictions = logisticRegr.predict(X_test)
  score = logisticRegr.score(X_test, y_test)
  scores.append(score)
  #print("{} Score : {}".format(feature[i],score))
  cm = metrics.confusion_matrix(y_test, predictions)
  f1_score = metrics.f1_score(y_test,predictions)
  f1.append(f1_score)
  mat += cm
  
  #print(cm)
  idx = np.mean(scores)
  print("{} Score : {}".format(feature[i],round(idx, 3)))
  print("{} F1 Score : {}".format(feature[i],round(np.mean(f1), 3)))
  print(mat)

multifeat Score : 0.534
multifeat F1 Score : 0.694
[[  1. 124.]
 [  1. 142.]]
vader Score : 0.534
vader F1 Score : 0.695
[[  1. 249.]
 [  1. 285.]]
blob Score : 0.534
blob F1 Score : 0.695
[[  2. 373.]
 [  2. 427.]]
tweet_dir Score : 0.533
tweet_dir F1 Score : 0.694
[[  3. 497.]
 [  4. 568.]]
tweet_volume Score : 0.533
tweet_volume F1 Score : 0.694
[[  3. 622.]
 [  4. 711.]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
data.head(2)

,Unnamed: 0,time,closed_price_by_hour,price_diff,movement,Daily_Weight_mean_by_hour,Daily_Weight_count_by_hour,blob_sent_mean_by_hour,subjectivity_mean_by_hour,vader_sent_mean_by_hour,tweet_diff,blob_sent_mean_by_hour_diff,vader_sent_mean_by_hour_diff,tweet_movement,blob_sent_movement,vader_sent_movement
0,0,2016-02-19 08:00:00,418.409535,1.745839,1,0.0,1,0.000000,0.15,-0.4019,0.0,-3.931830,-0.442529,1,0,0
1,1,2016-05-15 21:00:00,456.989825,-0.042136,0,1.0,1,0.136364,0.50,0.0000,0.0,0.136364,0.401900,1,1,1


In [ ]:
from sklearn import neighbors
scores = []
k_neighbours = []
feat = data.filter(['closed_price_by_hour','vader_sent_mean_by_hour']).values
for i in range(0,len(features)):
    df_feature = pd.DataFrame(features[i])
    df_label = pd.DataFrame(diff)
    for j in range(3,6):
        for weights in ["distance"]:
            print(weights,j)
            print("Working on {}".format(feature[i]))
            if i == 0:
              x_train, x_test, y_train, y_test = train_test_split(multifeat, df_label, test_size=0.20, random_state=0)
            #sbar = features[i].reshape(-1,1)
            else:    
              x_train, x_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=0.20, random_state=0)
            knn = neighbors.KNeighborsClassifier(j, weights=weights)
            knn.fit(x_train, y_train)
            predictions = knn.predict(x_test)
            score = knn.score(x_test,y_test)
            print("The score is : {}".format(round(score, 3)))
            cm = metrics.confusion_matrix(y_test,predictions)
            print(cm)
            f1 =  metrics.f1_score(y_test,predictions)
            print("The F1 score is : {} \n".format(round(f1, 3)))

distance 3
Working on multifeat
The score is : 0.519
[[66 54]
 [75 73]]
The F1 score is : 0.531 

distance 4
Working on multifeat
The score is : 0.522
[[67 53]
 [75 73]]
The F1 score is : 0.533 

distance 5
Working on multifeat
The score is : 0.56
[[71 49]
 [69 79]]
The F1 score is : 0.572 

distance 3
Working on vader
The score is : 0.507
[[ 95  25]
 [107  41]]
The F1 score is : 0.383 

distance 4
Working on vader
The score is : 0.515
[[ 96  24]
 [106  42]]
The F1 score is : 0.393 

distance 5
Working on vader
The score is : 0.522
[[ 94  26]
 [102  46]]
The F1 score is : 0.418 

distance 3
Working on blob
The score is : 0.5
[[69 51]
 [83 65]]
The F1 score is : 0.492 

distance 4
Working on blob
The score is : 0.5
[[71 49]
 [85 63]]
The F1 score is : 0.485 

distance 5
Working on blob
The score is : 0.496
[[68 52]
 [83 65]]
The F1 score is : 0.491 

distance 3
Working on tweet_dir
The score is : 0.549
[[  5 115]
 [  6 142]]
The F1 score is : 0.701 

distance 4
Working on tweet_dir
The 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed

The F1 score is : 0.704 

distance 3
Working on tweet_volume
The score is : 0.493
[[ 21  99]
 [ 37 111]]
The F1 score is : 0.62 

distance 4
Working on tweet_volume
The score is : 0.496
[[ 22  98]
 [ 37 111]]
The F1 score is : 0.622 

distance 5
Working on tweet_volume
The score is : 0.493
[[ 21  99]
 [ 37 111]]
The F1 score is : 0.62 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

features = [multifeat, vader,blob,tweetdir,tweet_vol,v_move,b_move]

feature = ['multifeat', 'vader','blob','tweetdir','tweet_vol','v_move','b_move']
for i in range(len(features)):
    scores = []
    mat = np.zeros([2,2])
    df_feature = pd.DataFrame(features[i])
    df_label = pd.DataFrame(diff)
    for j in range(1,10):
        print("Working on {} {} estimators".format(feature[i],j))
        #sbar = features[i].reshape(-1,1)

        if i == 0:
          #x_train, x_test, y_train, y_test = train_test_split(sbar, data["movement"], test_size=0.20, random_state=0)
          x_train, x_test, y_train, y_test = train_test_split(multifeat, df_label, test_size=0.20, random_state=0)
        else:
          x_train, x_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=0.20, random_state=0)
        regressor = RandomForestClassifier(n_estimators=j*10, random_state=0,max_depth=i+2)
        regressor.fit(x_train, y_train)
        y_pred = regressor.predict(x_test)
        
        #predictions = knn.predict(x_test)
        score = regressor.score(x_test,y_test)
        #scores.append(score)
        print("Confusion Matrix is: \n{}".format(confusion_matrix(y_test,y_pred)))
        
        #print(classification_report(y_test,y_pred))
        print("Accuracy Score is: {}\n".format(round(accuracy_score(y_test, y_pred), 3)))
        f1 =  metrics.f1_score(y_test,y_pred)
        print("The F1 score is : {} \n".format(round(f1, 3)))
        #print(scores)
        #print(mat)
    #print("The score is : {}".format(score))
    #cm = metrics.confusion_matrix(y_test,predictions)
    #print(cm)

Working on multifeat 1 estimators
Confusion Matrix is: 
[[ 18 102]
 [ 12 136]]
Accuracy Score is: 0.575

The F1 score is : 0.705 

Working on multifeat 2 estimators
Confusion Matrix is: 
[[ 13 107]
 [ 12 136]]
Accuracy Score is: 0.556

The F1 score is : 0.696 

Working on multifeat 3 estimators
Confusion Matrix is: 
[[  9 111]
 [  7 141]]
Accuracy Score is: 0.56

The F1 score is : 0.705 

Working on multifeat 4 estimators


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[ 12 108]
 [ 10 138]]
Accuracy Score is: 0.56

The F1 score is : 0.701 

Working on multifeat 5 estimators
Confusion Matrix is: 
[[ 12 108]
 [  9 139]]
Accuracy Score is: 0.563

The F1 score is : 0.704 

Working on multifeat 6 estimators


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[ 14 106]
 [ 12 136]]
Accuracy Score is: 0.56

The F1 score is : 0.697 

Working on multifeat 7 estimators
Confusion Matrix is: 
[[ 14 106]
 [ 12 136]]
Accuracy Score is: 0.56

The F1 score is : 0.697 

Working on multifeat 8 estimators


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[ 23  97]
 [ 35 113]]
Accuracy Score is: 0.507

The F1 score is : 0.631 

Working on b_move 7 estimators
Confusion Matrix is: 
[[ 22  98]
 [ 33 115]]
Accuracy Score is: 0.511

The F1 score is : 0.637 

Working on b_move 8 estimators


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Confusion Matrix is: 
[[ 22  98]
 [ 35 113]]
Accuracy Score is: 0.504

The F1 score is : 0.63 

Working on b_move 9 estimators
Confusion Matrix is: 
[[ 21  99]
 [ 34 114]]
Accuracy Score is: 0.504

The F1 score is : 0.632 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
